<a href="https://colab.research.google.com/github/Kolo-Naukowe-Axion/QC1/blob/dnn/dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo
import numpy as np

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


Taking data, conversion to numpy

In [ ]:
banknote_authentication = fetch_ucirepo(id=267)

X_raw = banknote_authentication.data.features
y_raw = banknote_authentication.data.targets

X = X_raw.values
y = y_raw.values.ravel()

Test and train split

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# stadarization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)

print(f"Training Tensor Shape: {X_train_tensor.shape}")
print(f"Testing Tensor Shape: {X_test_tensor.shape}")

Training Tensor Shape: torch.Size([1097, 4])
Testing Tensor Shape: torch.Size([275, 4])


In [31]:
class BanknoteDNN(nn.Module):
    def __init__(self):
        super(BanknoteDNN, self).__init__()
        self.layer1 = nn.Linear(4, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 16)
        self.output = nn.Linear(16, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x

model = BanknoteDNN().to(device)

In [32]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 200
print("\nStarting training...")

for epoch in range(epochs):
    model.train()

    outputs = model(X_train_tensor)

    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test_tensor)
        test_loss = criterion(test_outputs, y_test_tensor)
        predicted = (test_outputs > 0.5).float()
        correct = (predicted == y_test_tensor).float().sum()
        accuracy = correct / y_test_tensor.shape[0]

    if epoch%10==0:
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {loss.item():.4f} | Test Loss: {test_loss.item():.4f} | Test Acc: {accuracy:.4f}")


Starting training...
Epoch 1/200 | Train Loss: 0.7086 | Test Loss: 0.7007 | Test Acc: 0.4618
Epoch 11/200 | Train Loss: 0.6720 | Test Loss: 0.6657 | Test Acc: 0.4909
Epoch 21/200 | Train Loss: 0.6266 | Test Loss: 0.6202 | Test Acc: 0.8073
Epoch 31/200 | Train Loss: 0.5652 | Test Loss: 0.5598 | Test Acc: 0.8727
Epoch 41/200 | Train Loss: 0.4764 | Test Loss: 0.4716 | Test Acc: 0.8873
Epoch 51/200 | Train Loss: 0.3591 | Test Loss: 0.3571 | Test Acc: 0.9200
Epoch 61/200 | Train Loss: 0.2419 | Test Loss: 0.2412 | Test Acc: 0.9382
Epoch 71/200 | Train Loss: 0.1511 | Test Loss: 0.1494 | Test Acc: 0.9600
Epoch 81/200 | Train Loss: 0.0982 | Test Loss: 0.0913 | Test Acc: 0.9855
Epoch 91/200 | Train Loss: 0.0662 | Test Loss: 0.0595 | Test Acc: 0.9855
Epoch 101/200 | Train Loss: 0.0560 | Test Loss: 0.0415 | Test Acc: 0.9855
Epoch 111/200 | Train Loss: 0.0431 | Test Loss: 0.0313 | Test Acc: 0.9964
Epoch 121/200 | Train Loss: 0.0329 | Test Loss: 0.0242 | Test Acc: 1.0000
Epoch 131/200 | Train Loss:

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

clf = LogisticRegression()
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
print(f"Simple model accuracy: {accuracy_score(y_test, preds)*100:.2f}%")

Simple model accuracy: 97.82%
